In [ ]:
### select the reginal catolog

In [ ]:
import pandas as pd
from datetime import datetime

def filter_by_lat_lon_time(
    file_path: str,
    lat_min: float,
    lat_max: float,
    lon_min: float,
    lon_max: float,
    start_time: str = None,
    end_time: str = None,
    output_path: str = "filtered_data.csv"
):
    """
    从 txt 文件中筛选出指定经纬度和时间范围内的数据，并保存为 CSV 文件。

    参数:
        file_path (str): 输入的 txt 文件路径。
        lat_min (float): 最小纬度。
        lat_max (float): 最大纬度。
        lon_min (float): 最小经度。
        lon_max (float): 最大经度。
        start_time (str, 可选): 起始时间（格式 'YYYY-MM-DD HH:MM:SS'）。
        end_time (str, 可选): 结束时间（格式 'YYYY-MM-DD HH:MM:SS'）。
        output_path (str): 输出的 CSV 文件路径。

    返回:
        pandas.DataFrame: 筛选后的数据。
    """

    # 定义列名
    columns = [
        'year', 'month', 'day', 'hour', 'minute', 'second',
        'latitude', 'longitude', 'anything', 'mag_type', 'magnitude'
    ]

    # 读取 txt 文件（空格分隔）
    df = pd.read_csv(file_path, sep=r'\s+', names=columns)

    # 组合时间列
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])

    # 经纬度筛选
    filtered_df = df[
        (df['latitude'] >= lat_min) & (df['latitude'] <= lat_max) &
        (df['longitude'] >= lon_min) & (df['longitude'] <= lon_max)
    ]

    # 时间筛选
    if start_time:
        start_dt = pd.to_datetime(start_time)
        filtered_df = filtered_df[filtered_df['datetime'] >= start_dt]

    if end_time:
        end_dt = pd.to_datetime(end_time)
        filtered_df = filtered_df[filtered_df['datetime'] <= end_dt]

    # 保存为 CSV 文件
    filtered_df.to_csv(output_path, index=False)

    print(f"✅ 已将筛选结果保存到 {output_path}")
    return filtered_df

In [ ]:
result = filter_by_lat_lon(
    file_path="/Users/chouyuhin/Desktop/CENCcat-format.txt",
    lat_min=26,
    lat_max=26.4,
    lon_min=102.15,
    lon_max=103.55,
    start_time="2020-01-01 00:00:00",
    end_time="2021-12-31 23:59:59",
    output_path="region3_cal.csv"
)

print(result.head())


✅ 已将筛选结果保存到 region3_cal.csv
      year  month  day  hour  minute  second  latitude  longitude  anything  \
1644  1970      3    3     8      22    44.0     26.25     103.15         0   
1779  1970      3   16     7      44    58.0     26.03     102.45         0   
1780  1970      3   16     7      46    46.0     26.05     102.48         0   
1785  1970      3   16    11      51    32.0     26.03     102.48         0   
1792  1970      3   17     4      47     3.0     26.30     102.20         0   

     mag_type  magnitude  
1644       ML        2.1  
1779       ML        3.4  
1780       ML        3.4  
1785       ML        2.3  
1792       ML        1.9  


### 拼接standard catalog和我的

In [5]:
import pandas as pd
import numpy as np
from seismostats import Catalog

data = pd.read_csv(
    '/Users/chouyuhin/Desktop/hypoDD_v2_1329_withmag_modified_final.reloc',
    delim_whitespace=True,
    usecols=['lat', 'lon', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second', 'magnitude']
)

data['time'] = pd.to_datetime(data[['year', 'month', 'day', 'hour', 'minute', 'second']])
# print(data['time']) 
data = data.rename(columns={'lat': 'latitude', 'lon': 'longitude'})
data = data[['longitude', 'latitude', 'depth', 'time', 'magnitude']]
data = data[data['time']>='2022-12-01 00:00:00']
print(len(data))
standard= pd.read_csv(
    '/Users/chouyuhin/Desktop/standard_catalog_QJ_220825_230313.csv',
    sep=',',
    usecols=['year', 'month', 'day', 'hour', 'minute', 'second','latitude', 'longitude','depth', 'mag_type', 'magnitude','time']
 
)
standard = standard[['longitude', 'latitude', 'depth', 'time', 'magnitude']]
standard = standard[standard['time']<='2022-12-01 00:00:00']
print(len(standard))
combined_data = pd.concat([data, standard], ignore_index=True)
# combined_data = combined_data.sort_values(by='time').reset_index(drop=True)
combined_data.to_csv('/Users/chouyuhin/Desktop/combined_test.csv', index=False)


1455
237


/var/folders/y4/__h0s0zj0qb4gjkjzynnyrcr0000gn/T/ipykernel_92961/2356758502.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(


### 将三个模型得到的reloc文件合并到一个

In [8]:
import pandas as pd
import glob

# 假设你的 6 个 reloc 文件都放在同一个目录下，并且以 .reloc 结尾
files = [
    '/Users/chouyuhin/workhome/QJvalidation/eqt_2023raw/hypoDD_eqt2023raw_withmag.reloc',
         '/Users/chouyuhin/workhome/QJvalidation/phasenet_2023raw/hypoDD_phasenet2023raw_withmag.reloc',
         '/Users/chouyuhin/workhome/QJvalidation/tran_2023raw/hypoDD_tran2023raw_withmag.reloc',
         '/Users/chouyuhin/workhome/QJvalidation/eqt_20228_11_raw/hypoDD_eqt2208_11raw_withmag.reloc',
         '/Users/chouyuhin/Desktop/hypoDD_tran2208_11raw_withmag.reloc',
         '/Users/chouyuhin/Desktop/hypoDD_tran2212raw_withmag.reloc']

# 指定列名
cols = ['lat', 'lon', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second', 'magnitude']

# 读取并合并所有文件
df_list = []
for f in files:
    df = pd.read_csv(f, sep=r"\s+", usecols=cols, engine='python')
    df_list.append(df)

# 合并为一个总表
data = pd.concat(df_list, ignore_index=True)

# 创建时间列
data['time'] = pd.to_datetime(
    data[['year', 'month', 'day', 'hour', 'minute', 'second']]
)

# 按 time 升序排序
data = data.sort_values('time').reset_index(drop=True)
# data = data[data['magnitude'] != -1.0]
# 导出为新的 reloc 文件
data.to_csv("merged_reloc.reloc", sep="\t", index=False)

### 处理，去掉重复记录

In [ ]:
import pandas as pd

# 读取文件
df = pd.read_csv("/Users/chouyuhin/_Workhome/Proj_Seismicgap/QJvalidation/hypoDD_final_allevents.reloc", sep="\t")

# 转换 time 列
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# 强制转换所有数值列为 float（关键）
num_cols = ['lat','lon','depth','year','month','day','hour','minute','second','magnitude']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# 拆分 good 与 bad
bad = df[df['magnitude'] == -1.0].copy()
good = df[df['magnitude'] != -1.0].copy()

good = good.sort_values('time')
bad['to_delete'] = False

# 遍历 bad 中每条记录
for idx, row in bad.iterrows():
    t0 = row['time']
    d0 = row['depth']

    # 时间窗口 ±1秒
    mask_time = (good['time'] >= t0 - pd.Timedelta(seconds=3)) & \
                (good['time'] <=  t0 + pd.Timedelta(seconds=3))
    candidates = good[mask_time]

    if not candidates.empty:
        # 深度差 ≤ 1 km
        mask_depth = (candidates['depth'] - d0).abs() <= 5.0
        if mask_depth.any():
            bad.loc[idx, 'to_delete'] = True

# 删除 bad 行
bad_clean = bad[~bad['to_delete']]

# 合并
df_clean = pd.concat([good, bad_clean], ignore_index=True)

# 排序
df_clean = df_clean.sort_values('time').reset_index(drop=True)

# 保存最终结果
df_clean.to_csv("merged_reloc_clean.reloc", sep="\t", index=False)

print("完成：已删除 magnitude=-1 且时间/深度重复的行。")

完成：已删除 magnitude=-1 且时间/深度重复的行。
